# DataFrame Operations

In [99]:
from pyspark.sql import Row

row = Row(name="Alice", age=11)
print(row)
print(row['name'], row['age'])
print(row.name, row.age)

row = Row(name="Alice", age=11, count=1)
print(row.count)
print(row['count'])

Row(name='Alice', age=11)
Alice 11
Alice 11
<built-in method count of Row object at 0x7fad897bf180>
1


In [100]:
# Data file at https://www.cse.ust.hk/msbd5003/data/building.csv
df = spark.read.csv('../data/building.csv', header=True, inferSchema=True)

In [107]:
# show the content of the dataframe
df.show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|       

In [102]:
# Print the dataframe schema in a tree format
df.printSchema()

root
 |-- BuildingID: integer (nullable = true)
 |-- BuildingMgr: string (nullable = true)
 |-- BuildingAge: integer (nullable = true)
 |-- HVACproduct: string (nullable = true)
 |-- Country: string (nullable = true)



In [108]:
# Retrieve specific columns from the dataframe
#df.select('BuildingID', 'Country').show(3)
df.select('BuildingAge').show()

+-----------+
|BuildingAge|
+-----------+
|         25|
|         27|
|         28|
|         17|
|          3|
|          9|
|         13|
|         25|
|         11|
|         23|
|         14|
|         26|
|         25|
|         17|
|         19|
|         23|
|         11|
|         25|
|         14|
|         19|
+-----------+



In [62]:
# Use GroupBy clause with dataframe 
df.groupBy('HVACProduct').count().show(3) #same as GROUPBY count

+-----------+-----+
|HVACProduct|count|
+-----------+-----+
|    ACMAX22|    4|
|     AC1000|    4|
|     JDNS77|    4|
+-----------+-----+
only showing top 3 rows



In [8]:
# Create an RDD from the dataframe
dfrdd = df.rdd
dfrdd.take(3)

[Row(BuildingID=1, BuildingMgr='M1', BuildingAge=25, HVACproduct='AC1000', Country='USA'),
 Row(BuildingID=2, BuildingMgr='M2', BuildingAge=27, HVACproduct='FN39TG', Country='France'),
 Row(BuildingID=3, BuildingMgr='M3', BuildingAge=28, HVACproduct='JDNS77', Country='Brazil')]

In [63]:
from pyspark.sql.functions import *
df.where("Country<'USA'").select('BuildingID', lit('OK')).show(3) #add a new column using lit

+----------+---+
|BuildingID| OK|
+----------+---+
|         2| OK|
|         3| OK|
|         4| OK|
+----------+---+
only showing top 3 rows



# Rewriting SQL with DataFrame API

In [10]:
dfCustomer = spark.read.csv('../data/Customer.csv', header=True, inferSchema=True)
dfProduct = spark.read.csv('../data/Product.csv', header=True, inferSchema=True)
dfDetail = spark.read.csv('../data/SalesOrderDetail.csv', header=True, inferSchema=True)
dfHeader = spark.read.csv('../data/SalesOrderHeader.csv', header=True, inferSchema=True)

In [1]:
#dfCustomer.printSchema()
#dfProduct.printSchema()
#dfDetail.printSchema()
#dfDetail.printSchema()

SELECT "ProductID", "Name", "ListPrice" 
FROM Product 
WHERE Color = 'black'

In [66]:
# use filter/where
#dfProduct#.filter("Color = 'Black'")\ 
#         .select('ProductID', 'Name', 'ListPrice')\
#         .show(truncate=False)
dfProduct.where(dfProduct.Color=='Black')\
         .select('ProductID', 'Name', 'ListPrice')\
         .show(3, truncate=False)

+---------+-------------------------+---------+
|ProductID|Name                     |ListPrice|
+---------+-------------------------+---------+
|680      |HL Road Frame - Black, 58|1431.5   |
|708      |Sport-100 Helmet, Black  |34.99    |
|722      |LL Road Frame - Black, 58|337.22   |
+---------+-------------------------+---------+
only showing top 3 rows



Transform on price, times 2.

In [67]:
# without without giving a newname
dfProduct.where(dfProduct.Color=='Black')\
         .select(dfProduct.ProductID, dfProduct['Name'], dfProduct['ListPrice'] * 2)\
         .show(3, truncate=False)

+---------+-------------------------+---------------+
|ProductID|Name                     |(ListPrice * 2)|
+---------+-------------------------+---------------+
|680      |HL Road Frame - Black, 58|2863.0         |
|708      |Sport-100 Helmet, Black  |69.98          |
|722      |LL Road Frame - Black, 58|674.44         |
+---------+-------------------------+---------------+
only showing top 3 rows



In [68]:
# with giving a new name "DoublePrice"
dfProduct.where(dfProduct.Color=='Black')\
         .select(dfProduct.ProductID, dfProduct['Name'], (dfProduct['ListPrice'] * 2).alias('DoublPrice'))\
         .show(3, truncate=False)

+---------+-------------------------+----------+
|ProductID|Name                     |DoublPrice|
+---------+-------------------------+----------+
|680      |HL Road Frame - Black, 58|2863.0    |
|708      |Sport-100 Helmet, Black  |69.98     |
|722      |LL Road Frame - Black, 58|674.44    |
+---------+-------------------------+----------+
only showing top 3 rows



Give BLACK items with DoublePrice larger than 100

In [69]:
#PYTHON expression in where.
dfProduct.filter("Color = 'Black'")\
         .where(dfProduct['ListPrice'] * 2 > 100)\
         .select(dfProduct.ProductID, dfProduct['Name'], (dfProduct['ListPrice'] * 2).alias('DoublPrice'))\
         .show(3, truncate=False)

+---------+-------------------------+----------+
|ProductID|Name                     |DoublPrice|
+---------+-------------------------+----------+
|680      |HL Road Frame - Black, 58|2863.0    |
|722      |LL Road Frame - Black, 58|674.44    |
|723      |LL Road Frame - Black, 60|674.44    |
+---------+-------------------------+----------+
only showing top 3 rows



SELECT ProductID, Name, ListPrice 
FROM Product 
WHERE Color = 'black' 
ORDER BY ProductID

In [70]:
#orderby the name of column.
dfProduct.filter("Color = 'Black'")\
         .select('ProductID', 'Name', 'ListPrice')\
         .orderBy('ListPrice')\
         .show(3, truncate=False)

+---------+---------------------+---------+
|ProductID|Name                 |ListPrice|
+---------+---------------------+---------+
|860      |Half-Finger Gloves, L|24.49    |
|858      |Half-Finger Gloves, S|24.49    |
|859      |Half-Finger Gloves, M|24.49    |
+---------+---------------------+---------+
only showing top 3 rows



Find all orders and details on black product,
return the product SalesOrderID, SalesOrderDetailID, Name, UnitPrice, and OrderQty

SELECT SalesOrderID, SalesOrderDetailID, Name, UnitPrice, OrderQty 
FROM SalesLT.SalesOrderDetail, SalesLT.Product
WHERE SalesOrderDetail.ProductID = Product.ProductID AND Color = 'Black'

SELECT SalesOrderID, SalesOrderDetailID, Name, UnitPrice, OrderQty 
FROM SalesLT.SalesOrderDetail
JOIN SalesLT.Product ON SalesOrderDetail.ProductID = Product.ProductID
WHERE Color = 'Black'

Spark SQL supports natural joins

In [39]:
dfDetail.join(dfProduct, 'ProductID') \
        .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty') \
        .filter("Color='Black'") \
        .show(3)
# If we move the filter to after select, it still works.  Why?

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71938|            113295|Sport-100 Helmet,...|   29.994|       5|
|       71902|            112988|Sport-100 Helmet,...|   20.994|       4|
|       71797|            111082|Sport-100 Helmet,...|  20.2942|      12|
+------------+------------------+--------------------+---------+--------+
only showing top 3 rows



In [45]:
# This also works:

d1 = dfDetail.join(dfProduct, 'ProductID') \
             .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty')
d1.show(3)
d2 = d1.filter("Color = 'Black'")
d2.show(3)
#d2.explain()

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71938|            113278|Sport-100 Helmet,...|   20.994|       3|
|       71936|            113228|Sport-100 Helmet,...|   20.994|       1|
|       71902|            112980|Sport-100 Helmet,...|   20.994|       2|
+------------+------------------+--------------------+---------+--------+
only showing top 3 rows

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71938|            113295|Sport-100 Helmet,...|   29.994|       5|
|       71902|            112988|Sport-100 Helmet,...|   20.994|       4|
|       71797|            111082|Sport-100 Helmet,...|  20.2942|      12|
+------------

In [50]:
# SparkSQL performs optimization depending on whether intermediate dataframe are cached or not:

d1 = dfDetail.join(dfProduct, 'ProductID') \
             .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty')
d1.unpersist()
d1.show(3)
#d2 = d1.where("OrderQty >= 10")
d2 = d1.filter("Color = 'Black'")
d2.show(3)

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71938|            113278|Sport-100 Helmet,...|   20.994|       3|
|       71936|            113228|Sport-100 Helmet,...|   20.994|       1|
|       71902|            112980|Sport-100 Helmet,...|   20.994|       2|
+------------+------------------+--------------------+---------+--------+
only showing top 3 rows

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71784|            110794|Sport-100 Helmet,...|   20.994|      10|
|       71783|            110751|Sport-100 Helmet,...|   20.994|      10|
|       71797|            111082|Sport-100 Helmet,...|  20.2942|      12|
+------------

In [ ]:
# This will report an error:
# save to disk and read it back, now you cant find attribute color.
d1 = dfDetail.join(dfProduct, 'ProductID') \
             .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty')
d1.write.csv('temp.csv', mode = 'overwrite', header = True)
d2 = spark.read.csv('temp.csv', header = True, inferSchema = True)
d2.filter("Color = 'Black'").show()

In [ ]:
# This will report an error, too

d1 = dfDetail.join(dfProduct, 'ProductID') \
             .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty')
d2 = d1.filter(d1['Color'] == 'Black').show()
#d2 = d1.filter("Color = 'Black'").show() #Use SQL will work, d1 is a projection, can not passing col name.
# Because the parser will try to find a column named 'Color' in d1, which doesn't exist

Find all orders that include at least one black product, 
return the product SalesOrderID, Name, UnitPrice, and OrderQty

SELECT DISTINCT SalesOrderID
FROM SalesLT.SalesOrderDetail
JOIN SalesLT.Product ON SalesOrderDetail.ProductID = Product.ProductID
WHERE Color = 'Black'

In [55]:
dfDetail.join(dfProduct.filter("Color='Black'"), 'ProductID') \
        .select('SalesOrderID') \
        .distinct() \
        .show(3)

+------------+
|SalesOrderID|
+------------+
|       71902|
|       71832|
|       71915|
+------------+
only showing top 3 rows



How many colors in the products?

SELECT COUNT(DISTINCT Color)
FROM SalesLT.Product

In [72]:
dfProduct.select('Color').distinct().count()
# It's 1 more than standard SQL.  In standard SQL, COUNT() does not count NULLs.

10

Find the total price of each order, 
return SalesOrderID and total price (column name should be ‘totalprice’)

SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
FROM SalesLT.SalesOrderDetail
GROUP BY SalesOrderID

In [ ]:
dfDetail.select('*', (dfDetail.UnitPrice*dfDetail.OrderQty*(1-dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .groupBy('SalesOrderID').sum('netprice') \
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .show(3)

In [ ]:
Find the total price of each order where the total price > 10000

SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
FROM SalesLT.SalesOrderDetail
GROUP BY SalesOrderID
HAVING SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) > 10000

In [90]:
dfDetail.select('*', (dfDetail.UnitPrice*dfDetail.OrderQty*(1-dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .groupBy('SalesOrderID').sum('netprice')\
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .where('TotalPrice>1000')\
        .show(3)

+------------+------------------+
|SalesOrderID|        TotalPrice|
+------------+------------------+
|       71902|59894.209199999976|
|       71832|      28950.678108|
|       71915|1732.8899999999999|
+------------+------------------+
only showing top 3 rows



Find the total price on the black products of each order where the total price > 10000

SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
FROM SalesLT.SalesOrderDetail, SalesLT.Product
WHERE SalesLT.SalesOrderDetail.ProductID = SalesLT.Product.ProductID AND Color = 'Black'
GROUP BY SalesOrderID
HAVING SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) > 10000

In [111]:
dfDetail.select('*', (dfDetail.UnitPrice*dfDetail.OrderQty*(1-dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .join(dfProduct, 'ProductID')\
        .filter("Color='Black'")\
        .groupBy('SalesOrderID').sum('netprice')\
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .where('TotalPrice>1000')\
        .show(3)

+------------+------------------+
|SalesOrderID|        TotalPrice|
+------------+------------------+
|       71902|26677.883999999995|
|       71832|      16883.748108|
|       71831|          1712.946|
+------------+------------------+
only showing top 3 rows



For each customer, find the total quantity of black products bought.
Report CustomerID, FirstName, LastName, and total quantity

SELECT saleslt.customer.customerid, FirstName, LastName, sum(orderqty)
FROM saleslt.customer
LEFT OUTER JOIN 
(
saleslt.salesorderheader
JOIN saleslt.salesorderdetail
ON saleslt.salesorderdetail.salesorderid = saleslt.salesorderheader.salesorderid
JOIN saleslt.product
ON saleslt.product.productid = saleslt.salesorderdetail.productid and color = 'black'
)
ON saleslt.customer.customerid = saleslt.salesorderheader.customerid
GROUP BY saleslt.customer.customerid, FirstName, LastName
ORDER BY sum(orderqty) 

In [121]:
#left outer join
d1 = dfDetail.join(dfProduct, 'ProductID')\
             .where('Color = "Black"') \
             .join(dfHeader, 'SalesOrderID')\
             .groupBy('CustomerID').sum('OrderQty')
dfCustomer.join(d1, 'CustomerID', 'left_outer')\
          .select('CustomerID', 'FirstName', 'LastName', 'sum(OrderQty)')\
          .orderBy('sum(OrderQty)', ascending=False)\
          .show(3)

+----------+---------+------------+-------------+
|CustomerID|FirstName|    LastName|sum(OrderQty)|
+----------+---------+------------+-------------+
|     30050|  Krishna|Sunkammurali|           89|
|     29796|      Jon|      Grande|           65|
|     29957|    Kevin|         Liu|           62|
+----------+---------+------------+-------------+
only showing top 3 rows



# Quiz

In [3]:
df = spark.read.csv('../data/sales.csv', header=True, inferSchema=True)

In [124]:
df.printSchema()

root
 |-- Transaction_date: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Account_Created: string (nullable = true)
 |-- Last_Login: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [127]:
# Q1
#Find all distinct countries.
df.select('Country').distinct().count()

34

In [133]:
# Q2
#Find the Name and Price of sales records in Brazil.
df.where("Country = 'Brazil'").select('Name', 'Price').show()

+-------+-----+
|   Name|Price|
+-------+-----+
|Joachim| 1200|
|  Diana| 7500|
+-------+-----+



In [21]:
# Q3
#For each country, find the total Price.
df.groupBy('Country').sum('Price')\
  .withColumnRenamed('sum(Price)', 'TotalPrice')\
  .show(3)

+--------+----------+
| Country|TotalPrice|
+--------+----------+
|  Sweden|      8400|
|  Jersey|      1200|
|Malaysia|      1200|
+--------+----------+
only showing top 3 rows



In [136]:
# Q4
#List countries by their total Price in descending order.
df.groupBy('Country').sum('Price')\
  .withColumnRenamed('sum(Price)', 'TotalPrice')\
  .orderBy('TotalPrice', ascending = False)\
  .show(3)

+--------------+----------+
|       Country|TotalPrice|
+--------------+----------+
| United States|    350350|
|United Kingdom|     63600|
|        Canada|     42000|
+--------------+----------+
only showing top 3 rows



In [2]:
# Q5
#Redo Question 3, but replace the country names by their IDs.
#For each country, find the total Price.
df2 = spark.read.csv('../data/countries.csv', header=True, inferSchema=True)

In [7]:
df2.printSchema()

root
 |-- Country: string (nullable = true)
 |-- ID: integer (nullable = true)



In [34]:
df.join(df2, 'Country').groupBy('ID').sum('Price').withColumnRenamed('sum(Price)', 'TotalPrice').show(3)

+---+----------+
| ID|TotalPrice|
+---+----------+
| 31|      1200|
| 34|      2400|
| 28|      3600|
+---+----------+
only showing top 3 rows



In [46]:
# Q6
#Rewrite the PageRank example using DataFrame API.  Here is a skeleton of the code.  
#Your job is to fill in the missing part.  The data files can be downloaded at:
from pyspark.sql.functions import *

numOfIterations = 10

lines = spark.read.text("../data/pagerank_data.txt")
# You can also test your program on the follow larger data set:
# lines = spark.read.text("dblp.in")

a = lines.select(split(lines[0],' '))
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
outdegrees = links.groupBy('src').count()
ranks = outdegrees.select('src', lit(1).alias('rank'))

#for iteration in range(numOfIterations):
# FILL IN THIS PART

#ranks.orderBy(desc('rank')).show()

In [96]:
for iteration in range(10):
    normalized_ranks = ranks.join(outdegrees, 'src')\
                            .withColumn('w', col('rank')/col('count')).select('src', 'w')
    ranks = links.join(normalized_ranks, 'src')\
                .groupBy('dst')\
                .sum('w')\
                .withColumn('rank', 0.15+0.85*col('sum(w)'))\
                .withColumnRenamed('dst', 'src')\
                .select('src', 'rank')


In [97]:
ranks.orderBy(desc('rank')).show()

+---+------------------+
|src|              rank|
+---+------------------+
|  1|1.2982699838536758|
|  4|0.9999999999999998|
|  3|0.9999999999999997|
|  2|0.7017300161463238|
+---+------------------+

